## LR Scraper and Estimator

This notebook contains: 

1. code to scrape diagnostic likelihood ratios from theNNT.com and convert them to numerical form. 
2. code to generate prompts for large language models to estimate the likelihood ratios, then calls to the OpenAI API. 

The initial output is a spreadsheet called: nnt_lrs_with_estimated which contains: 
- a sheet for each diagnosis or prediction target
- a row for each piece of information
- columns for the name, raw nnt lr, processed nnt lr, and estimated by 1 or more LLMs

Then, there is a final processing step to collate the reported and estimated LRs into a single sheet to facilitate data analysis, manually named NNT_LRs_08-26-2025.xlsx. 

For manuscript quality reference standard data-set, manual cleaning supplemented this automated extraction pipeline (tasks noted in workbook). 

- The un-manually cleaned version is nnt_lrs_processed_without_manual.xlsx
- the manually cleaned version, which is used for the manuscript, is nnt_lrs_processed.xlsx

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import math
import os
import json
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI  # or your appropriate client wrapper
import math
from dotenv import load_dotenv

load_dotenv()  # looks for a .env file (in the current dir by default) that contains an OPENAI_API_KEY="" variable
#print(os.getenv("OPENAI_API_KEY"))  # don't upload your openai key to github.

### Extract Data from the NNT 

This scrapes all of the likelihood ratios listed on the NNT ('https://thennt.com/home-lr/') into Excel spreadsheets. 

1. A spreadsheet ("nnt_lrs.xlsx") contains a separate sheet for each page, which corresponds to a "prediction tasks" e.g. diagnosing the cause of a symptom - sometimes with specification of an intended population. Each sheet contains two columns: the name of the features (e.g. test result, finding, historical occurence, comorbditiy), the second contains the raw listing from the spreadsheet

2. A second spreadsheet contains the same sheets corresponding to a prediction target, and all of the features. These are used two call the 

In [ ]:
def get_specialty_links():
    """
    Extracts specialties and their corresponding article links from the webpage.
    Returns a list of dictionaries with specialty names and associated links.
    """

    url = 'https://thennt.com/home-lr/'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the section with "Diagnosis (LR) Reviews by Specialty"
    specialty_section = soup.find('div', class_='well subdisplay accordion_caption', id='lr-byspecialty')

    if not specialty_section:
        print("Could not find the 'Diagnosis (LR) Reviews by Specialty' section on the webpage.")
        return []

    results = []

    # Find all specialty headings (e.g., h3)
    subheadings = specialty_section.find_all('h3')

    for subheading in subheadings:
        subheading_text = subheading.get_text(strip=True)  # Get specialty name
        links = []

        # Find the next unordered list (ul) which contains links
        next_ul = subheading.find_next_sibling('ul')

        if next_ul:
            for a_tag in next_ul.find_all('a', href=True):
                link_text = a_tag.get_text(strip=True)  # Link display name
                link_href = a_tag['href']  # Actual URL
                links.append({'display_name': link_text, 'url': link_href})

        results.append({'specialty': subheading_text, 'links': links})

    return results

def extract_likelihood_ratios(page_content):
    """
    Parses all likelihood ratio tables within <article class="lr_cards_details">.
    For each subsection indicated by an <h3> heading:
      - If the heading indicates Positive Findings, each finding will be prefixed with "Patient has: ".
      - If the heading indicates Negative Findings, a leading "No" (if present) is removed from the finding and then it is prefixed with "Patient does not have: ".
    This function processes all tables under a given heading (i.e. until the next <h3> is reached).
    Returns a list of tuples: (finding, likelihood ratio).
    """
    from bs4 import BeautifulSoup
    import re

    soup = BeautifulSoup(page_content, 'html.parser')
    results = []
    
    # Locate the main LR details section.
    lr_section = soup.find('article', class_='lr_cards_details')
    if not lr_section:
        return results

    # Find all <h3> headings in the section.
    headings = lr_section.find_all('h3')
    
    if headings:
        for h3 in headings:
            heading_text = h3.get_text(strip=True)
            if "Positive Findings" in heading_text:
                prefix = "Patient has: "
            elif "Negative Findings" in heading_text:
                prefix = "Patient does not have: "
            else:
                prefix = ""
            
            # Process all sibling elements until the next <h3> is encountered.
            sibling = h3.find_next_sibling()
            while sibling and sibling.name != "h3":
                if sibling.name == "table" and "lrtable" in sibling.get("class", []):
                    # Try to get proper data rows (i.e. <tr> elements with <td>).
                    rows = sibling.find_all("tr")
                    data_rows = [row for row in rows if row.find_all("td")]
                    
                    if data_rows:
                        for row in data_rows:
                            cols = row.find_all("td")
                            if len(cols) >= 2:
                                finding = cols[0].get_text(strip=True)
                                lr_value = cols[1].get_text(strip=True)
                                # If there's an <a> inside the LR cell, use its text.
                                link = cols[1].find("a")
                                if link:
                                    lr_value = link.get_text(strip=True) or lr_value
                                if not lr_value:
                                    lr_value = "Not reported"
                                
                                # Modify the finding string based on the prefix.
                                if prefix:
                                    if prefix.startswith("Patient does not have:"):
                                        finding = re.sub(r'^no\s+', '', finding, flags=re.IGNORECASE)
                                    finding = prefix + finding
                                
                                results.append((finding, lr_value))
                    else:
                        # If no rows with <td> are found, assume the table contains <td> elements in sequence.
                        all_tds = sibling.find_all("td")
                        # Process in pairs.
                        for i in range(0, len(all_tds), 2):
                            finding = all_tds[i].get_text(strip=True)
                            if i+1 < len(all_tds):
                                lr_value = all_tds[i+1].get_text(strip=True)
                            else:
                                lr_value = "Not reported"
                            # Check for an <a> element.
                            a_tag = all_tds[i+1].find("a")
                            if a_tag:
                                lr_value = a_tag.get_text(strip=True) or lr_value
                            if not lr_value:
                                lr_value = "Not reported"
                            
                            if prefix:
                                if prefix.startswith("Patient does not have:"):
                                    finding = re.sub(r'^no\s+', '', finding, flags=re.IGNORECASE)
                                finding = prefix + finding
                            results.append((finding, lr_value))
                sibling = sibling.find_next_sibling()
    else:
        # Fallback: process all tables in the section if no <h3> headings exist.
        tables = lr_section.find_all('table', class_='lrtable')
        for table in tables:
            rows = table.find_all("tr")
            data_rows = [row for row in rows if row.find_all("td")]
            if data_rows:
                for row in data_rows:
                    cols = row.find_all("td")
                    if len(cols) >= 2:
                        finding = cols[0].get_text(strip=True)
                        lr_value = cols[1].get_text(strip=True)
                        link = cols[1].find("a")
                        if link:
                            lr_value = link.get_text(strip=True) or lr_value
                        if not lr_value:
                            lr_value = "Not reported"
                        results.append((finding, lr_value))
            else:
                all_tds = table.find_all("td")
                for i in range(0, len(all_tds), 2):
                    finding = all_tds[i].get_text(strip=True)
                    if i+1 < len(all_tds):
                        lr_value = all_tds[i+1].get_text(strip=True)
                    else:
                        lr_value = "Not reported"
                    a_tag = all_tds[i+1].find("a") if i+1 < len(all_tds) else None
                    if a_tag:
                        lr_value = a_tag.get_text(strip=True) or lr_value
                    if not lr_value:
                        lr_value = "Not reported"
                    results.append((finding, lr_value))
                    
    return results

def fetch_webpages(specialty_links):
    """
    Iterates through all the extracted links, fetches the webpage content, 
    and extracts likelihood ratio findings.
    """
    findings_by_display_name = {}

    for item in specialty_links:
        print(f"Fetching pages for Specialty: {item['specialty']}")

        for link in item['links']:
            display_name = link['display_name']
            url = link['url']

            try:
                print(f"  - Fetching: {display_name} ({url})")
                response = requests.get(url)

                if response.status_code == 200:
                    print(f"    Success: {display_name} page fetched.")
                    
                    # Extract likelihood ratio findings
                    findings = extract_likelihood_ratios(response.text)
                    
                    # Store the extracted data
                    findings_by_display_name[display_name] = findings

                else:
                    print(f"    Failed to fetch {display_name} - Status Code: {response.status_code}")

                time.sleep(1)  # Optional: Add a delay to avoid overwhelming the server

            except requests.RequestException as e:
                print(f"    Error fetching {display_name}: {e}")

        print("\n")  # Add space between specialties for readability

    return findings_by_display_name

def save_to_excel(findings_data, filename="nnt_lrs.xlsx", blank_values=False):
    """
    Saves likelihood ratios to an Excel file with each display_name as a separate sheet.
    If blank_values is True, the Likelihood Ratio column is left blank.
    The first row contains the full display_name, and column headers start from the second row.
    """
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for display_name, findings in findings_data.items():
            if findings:
                # Prepare DataFrame
                df = pd.DataFrame(findings, columns=["Finding", "Likelihood Ratio"])

                if blank_values:
                    df["Likelihood Ratio"] = ""  # Clear likelihood ratio values

                # Insert full display_name as the first row
                full_name_row = pd.DataFrame({df.columns[0]: [display_name], df.columns[1]: [""]})
                df = pd.concat([full_name_row, df], ignore_index=True)

                # Save to Excel with sheet name as the **last** 31 characters
                sheet_name = display_name[-31:]
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)  # No default header

            else:
                print(f"Skipping {display_name} (No data found).")

    print(f"\nLikelihood ratios saved to {filename}")

# Fetch specialties and links
specialty_links = get_specialty_links()
findings_data = fetch_webpages(specialty_links)

# Save normal file
save_to_excel(findings_data, "nnt_lrs.xlsx", blank_values=False)

# Save version with blank likelihood ratios
save_to_excel(findings_data, "nnt_lrs_sans_number.xlsx", blank_values=True)

### Convert LR's to numerical format

This block takes in the excel spreadsheet with raw data from theNNT.com ("nnt_lrs.xlsx") and creates a new spreadsheet ("nnt_lrs_processed.xlsx") with a third column that contains a numerical version of the second column (raw data from theNNT) to be used as the LR_llm.

It removes any 'x's from the input, then determines whether the cell reports 

1. point estimate only (in which case use the point estimate)
2. point estimate + range (in which case take the point estimate), or 
3. range only (in which case, calculate the geometric mean)

It also counts the number of conditions (last 31 letters due to excel limitation) and LRs

[ ] TODO / Issues list: 
- some of the BNP thresholds just have a number rather than a specification of the full "BNP > 100"; need to use header e.g. https://thennt.com/lr/dyspnea-due-to-heart-failure-without-chronic-respiratory-disease/ and https://thennt.com/lr/dyspnea-due-to-heart-failure-without-chronic-respiratory-disease/ - for the manuscript, these had to be manually reconciled.
- some of the tables on the NNT contain categories that define the type of finding (ie 'edema' in a 'Chest X-ray findings') - these are manually reconciled/cleaned up for the manuscript as well. 
- some of the "Prediction Targets" contain specifications not relevant to the task (ie. page title = "Accuracy of Ultrasound Findings for Diagnosing Retinal Pathology.." but prediction target is Retinal Pathology). This is also manually cleaned for the manuscript.


In [ ]:
def parse_lr(lr_str):
    """
    Given a string from the 'Likelihood Ratio' cell, this function:
      - Removes any 'x' characters from the input.
      - If the string contains a parenthesized range (i.e. a point estimate plus a range),
        it returns the point estimate.
      - If the entire string is a range (e.g. "0.92-1.1", "3.3 to 4.8", "4.8–7.6"),
        it computes and returns the geometric mean.
      - Otherwise, it returns a float based on the first number found.
      - If the value is missing or cannot be parsed, returns NaN.
    """
    # Remove all 'x' characters and trim whitespace
    lr_str = lr_str.replace("x", "").strip()
    if lr_str == "":
        return np.nan

    # If parentheses exist, assume format "point_estimate (range)" and use the point estimate.
    if "(" in lr_str:
        point_part = lr_str.split("(")[0].strip()
        try:
            return float(point_part)
        except Exception:
            pass

    # Check for a range-only pattern.
    # This regex looks for two numbers separated by "to", "-" or "–" with optional whitespace.
    range_only_match = re.match(r'^\s*([0-9]*\.?[0-9]+)\s*(to|[-–])\s*([0-9]*\.?[0-9]+)\s*$', lr_str)
    if range_only_match:
        try:
            low = float(range_only_match.group(1))
            high = float(range_only_match.group(3))
            return math.sqrt(low * high)
        except Exception:
            return np.nan

    # Fallback: if no range-only pattern is found, extract the first number and return it.
    numbers = re.findall(r'([0-9]*\.?[0-9]+)', lr_str)
    if numbers:
        try:
            return float(numbers[0])
        except Exception:
            return np.nan

    return np.nan

# Load the original Excel file (each sheet has no header and the first row is the display name row)
input_filename = "nnt_lrs.xlsx"
output_filename = "nnt_lrs_processed.xlsx"

# Read all sheets from the Excel file into a dictionary of DataFrames.
excel_sheets = pd.read_excel(input_filename, sheet_name=None, header=None)

total_lr_count = 0
sheet_counts = {}

with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    for sheet_name, df in excel_sheets.items():
        numerical_lr = []
        # Process each row in the sheet.
        for idx, row in df.iterrows():
            # For the header row (assumed to be the first row: condition label), add an empty string.
            if idx == 0:
                numerical_lr.append("")
            else:
                cell_value = row[1]  # The original "Likelihood Ratio" is in the second column (index 1)
                if pd.isna(cell_value) or str(cell_value).strip() == "":
                    numerical_lr.append("")
                else:
                    numerical_lr.append(parse_lr(str(cell_value)))
        
        # Insert the new column immediately after the "Likelihood Ratio" column.
        # This makes the new column the third column.
        df.insert(2, "Numerical LR", numerical_lr)
        
        # Remove rows (except the header) where the new "Numerical LR" is empty or NaN.
        header = df.iloc[[0]]  # Keep the header row (the condition label)
        data = df.iloc[1:]
        data = data[data["Numerical LR"].apply(lambda x: not (x == "" or pd.isna(x)))]
        filtered_df = pd.concat([header, data], ignore_index=True)
        
        # Insert a new row (after the condition label row) with the column labels.
        # The final sheet will have:
        #   Row 0: Condition label (from the original sheet)
        #   Row 1: Column labels: 'finding', 'lr_raw', and 'lr_num'
        #   Row 2+: Data rows
        col_labels = pd.DataFrame([["finding", "lr_raw", "lr_reported"]], columns=filtered_df.columns)
        final_df = pd.concat([filtered_df.iloc[[0]], col_labels, filtered_df.iloc[1:]], ignore_index=True)
        
        # Count the number of LR values for this sheet (exclude the two header rows).
        lr_count = len(final_df) - 2
        sheet_counts[sheet_name] = lr_count
        total_lr_count += lr_count
        
        # Write the modified DataFrame to the new Excel file.
        # The output maintains the original format: no index and no additional header row.
        final_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

# Display counts.
num_sheets = len(excel_sheets)
print(f"Processed {num_sheets} condition(s) (sheets).")
for sheet, count in sheet_counts.items():
    print(f"Sheet '{sheet}' has {count} LR value(s).")
print(f"Total LR values processed across all sheets: {total_lr_count}.")

print(f"Processed Excel file saved as '{output_filename}'")

## Estimate LRs

This code block reads in the data from the nnt_lr_processed.xlsx excel file and calls a list of openAI models to have them give there best (single) estimate of the LR. Then, it rights a new spreadsheet nnt_lr_estimates that includes columns in each spreadsheet for each estimation. 

## Newest Version (Aug 22nd)

GPT-5 Capabilities added and revised prompting strategy mildly.

In [ ]:
"""
Bayesian LR estimator — Responses API
Updated: 2025‑08‑22

Changes vs. your prior block:
• Responses API only (no Chat fallback).
• No max_output_tokens (output is schema‑constrained and tiny).
• Reasoning models use reasoning.effort="medium"; no temperature/top_p.
• GPT‑5(+mini) use text.verbosity="low" (applied only where supported).
• Includes these models: gpt‑5, gpt‑5‑mini, gpt‑4o‑mini‑2024‑07‑18, gpt‑4o‑2024‑08‑06,
  gpt‑4.1‑mini‑2025‑04‑14, gpt‑4.1‑2025‑04‑14, o3‑mini‑2025‑01‑31, o3‑2025‑04‑16, o4‑mini‑2025‑04‑16.
"""
from __future__ import annotations

import os
import logging
from pathlib import Path
from typing import Optional
import time, math
from random import random

import pandas as pd
from pydantic import BaseModel
from openai import OpenAI

# -----------------------------------------------------------------------------
# 0) Configuration
# -----------------------------------------------------------------------------
logging.basicConfig(level=logging.WARNING)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Model registry: 
MODEL_CAPABILITIES = {
    # GPT‑5 series (reasoning; supports text.verbosity; no temperature)
    "gpt-5"        : {"reasoning": True,  "verbosity": True,  "allow_temp": False},
    "gpt-5-mini"   : {"reasoning": True,  "verbosity": True,  "allow_temp": False},
    "gpt-5-nano"   : {"reasoning": True,  "verbosity": True,  "allow_temp": False},

    # GPT‑4.1 family (non‑reasoning; temperature OK); include snapshots + aliases
    "gpt-4.1-2025-04-14" : {"reasoning": False, "verbosity": False, "allow_temp": True},
    "gpt-4.1-mini-2025-04-14": {"reasoning": False, "verbosity": False, "allow_temp": True},
    "gpt-4.1-nano-2025-04-14": {"reasoning": False, "verbosity": False, "allow_temp": True},

    # GPT‑4o family (non‑reasoning; temperature OK); prefer latest snapshot or alias
    "gpt-4o-2024-11-20"  : {"reasoning": False, "verbosity": False, "allow_temp": True},
    "gpt-4o-mini-2024-07-18": {"reasoning": False, "verbosity": False, "allow_temp": True},

    # o‑series (reasoning; no temperature)
    "o3-2025-04-16" : {"reasoning": True,  "verbosity": False, "allow_temp": False},
    "o3-mini-2025-01-31": {"reasoning": True,  "verbosity": False, "allow_temp": False},
    "o4-mini-2025-04-16": {"reasoning": True,  "verbosity": False, "allow_temp": False},
}
MODELS = list(MODEL_CAPABILITIES)

# -----------------------------------------------------------------------------
INPUT_FILE  = "nnt_lrs_processed.xlsx"
OUTPUT_FILE = "nnt_lrs_with_estimated.xlsx"

# -----------------------------------------------------------------------------
# 1) Prompt 
# -----------------------------------------------------------------------------
SYSTEM_CORE = """You are a Bayesian diagnostic assistant.
Estimate a numeric likelihood ratio (LR) for a finding with respect to a diagnosis.
Return only a JSON object matching the schema: {"value": <float>}, where value > 0.
"""

DEFINITION = """Definition:
LR = P(finding | diagnosis) / P(finding | not-diagnosis)
"""

BANDS = """LR evidence bands (reference):
>10 strong for; 5-10 moderate for; 2–5 weak for;
0.5–2 negligible;
0.2-0.5 weak against; 0.1-0.2 moderate against; ≤0.1 strong against"""

# Few‑shot examples - these are human guestimates (to avoid seeding the dataset and inflating performance)
FEW_SHOT_RICH = [
    ("deep vein thrombosis",    "femoral vein noncompressaible on ultrasound",      16.0), # some data this might be higher? 
    ("pericarditis",            "pleuritic chest pain improved by leaning forward",  5.2),
    ("pulmonary embolism",      "tachycardia >100 bpm",                              2.2),
    ("urinary tract infection", "malodorous urine",                                  1.1),
    ("myocardial infarction",  "enjoys playing chess",                               1.0),
    ("appendicitis",            "no RLQ tenderness",                                0.45),
    ("pneumothorax",            "bilateral lung sliding present on US",             0.18), # some data this might be lower?
    ("HIV infection",           "4th‑generation Ag/Ab screen negative beyond window",0.05),

]

FEW_SHOT_MIN = [
    ("deep vein thrombosis",    "femoral vein noncompressaible on ultrasound",     16.0),
    ("myocardial infarction",  "enjoys playing chess",                             1.0),
]

def build_messages(diagnosis: str, finding: str, reasoning: bool) -> list[dict]:
    msgs: list[dict] = [
        {"role": "system", "content": SYSTEM_CORE.strip()},
        {"role": "system", "content": DEFINITION.strip()},
        {"role": "system", "content": BANDS.strip()},
    ]
    examples = FEW_SHOT_MIN if reasoning else FEW_SHOT_RICH
    for dx_ex, f_ex, v_ex in examples:
        msgs.append({"role": "user",      "content": f"Condition: {dx_ex}\nFinding: {f_ex}"})
        msgs.append({"role": "assistant", "content": f'{{"value": {float(v_ex)}}}'})
    msgs.append({"role": "user", "content": f"Condition: {diagnosis}\nFinding: {finding}"})
    return msgs

# -----------------------------------------------------------------------------
# 2) Structured Outputs schema (Pydantic)
# -----------------------------------------------------------------------------
class LRResponse(BaseModel):
    value: float

# -----------------------------------------------------------------------------
# 2b) Retry wrapper (exponential backoff with jitter)
# -----------------------------------------------------------------------------
def estimate_lr_until_positive(
    diagnosis: str,
    finding: str,
    model: str,
    client: Optional[OpenAI] = None,
    max_retries: Optional[int] = None,      # None ⇒ retry indefinitely
    base_backoff: float = 0.5,              # seconds
    max_backoff: float = 30.0               # seconds
) -> float:
    attempt = 0
    while True:
        attempt += 1
        try:
            lr = estimate_lr(diagnosis, finding, model, client)
            if isinstance(lr, (int, float)) and math.isfinite(lr) and lr > 0:
                return float(lr)
            raise ValueError(f"Non‑positive or non‑finite LR: {lr!r}")
        except Exception as e:
            logging.warning(
                f"[retry {attempt}] sheet finding='{finding[:80]}' | "
                f"model={model} → {e}"
            )
            if (max_retries is not None) and (attempt >= max_retries):
                raise
            # exponential backoff with jitter
            delay = min(base_backoff * (2 ** (attempt - 1)), max_backoff)
            time.sleep(delay * (0.5 + random()))  # 0.5–1.5× jitter

# -----------------------------------------------------------------------------
# 3) Estimator call (Responses API)
# -----------------------------------------------------------------------------
def estimate_lr(diagnosis: str, finding: str, model: str, client: Optional[OpenAI] = None) -> float:
    if client is None:
        client = OpenAI()

    cfg = MODEL_CAPABILITIES[model]
    msgs = build_messages(diagnosis, finding, reasoning=cfg["reasoning"])

    kwargs = {}
    if cfg["reasoning"]:
        kwargs["reasoning"] = {"effort": "medium"}     # for GPT‑5 and o‑series
        # no temperature/top_p
    elif cfg["allow_temp"]:
        kwargs["temperature"] = 0.2                    # allowed for 4o / 4.1

    # Apply verbosity only where supported (GPT‑5 family)
    if cfg["verbosity"]:
        kwargs["text"] = {"verbosity": "low"}

    resp = client.responses.parse(
        model=model,
        input=msgs,
        text_format=LRResponse,    # Structured Outputs → Pydantic
        **kwargs,
    )
    return float(resp.output_parsed.value)

# -----------------------------------------------------------------------------
# 4) Main pipeline: read workbook → append model columns → write output
# -----------------------------------------------------------------------------
def run_batch(input_file: str | Path, output_file: str | Path, models: list[str]) -> None:
    sheets = pd.read_excel(input_file, sheet_name=None, header=None)

    for sheet_name, df in sheets.items():
        diagnosis = str(df.iloc[0, 0]).strip()
        for model in models:
            new_header = "lr_" + model
            col = []
            print(f"→ {diagnosis[:60]} | {model}")
            for i in range(len(df)):
                if i == 0:
                    col.append("")                   # top-left cell (sheet label row)
                elif i == 1:
                    col.append(new_header)           # column header row
                else:
                    finding = str(df.iloc[i, 0]).strip()
                    if not finding:
                        col.append("")               # keep blank rows blank
                        continue
                    try:
                        # retry until a strictly positive, finite float is returned
                        lr = estimate_lr_until_positive(
                            diagnosis, finding, model, client,
                            max_retries=None         # set to an int (e.g., 8) to cap retries
                        )
                    except Exception as e:
                        lr = "ERROR"
                        logging.warning(
                            f"Error on sheet '{sheet_name}', row {i}, model {model} after retries: {e}"
                        )
                    col.append(lr)
            # Insert as object dtype to accommodate strings like "ERROR"
            df.insert(df.shape[1], new_header, pd.Series(col, dtype="object"))
        sheets[sheet_name] = df

    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for name, frame in sheets.items():
            frame.to_excel(writer, sheet_name=name, index=False, header=False)

    print(f"Done – results saved to '{output_file}'")

if __name__ == "__main__":
    run_batch(INPUT_FILE, OUTPUT_FILE, MODELS)

→ Confirmation of Endotracheal Tube Placement | gpt-5


#### Collate the outputs into a single sheet 

Note: the automated classify_feature_type performance is low, thus the manuscript relies on manual classification of the feature type. 

In the call for the convert_lr_workbook, 3 files must be specified

- input_file = the output file from the above LR estimator loop. 
- output_file = where to write the output
- template_file = an example workbook that contains the desired output format (for specification)


In [ ]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------------ #
# 1.  ***Optional*** helper – keep the same “Feature Type” buckets   
#  NOTE: this does not seem like it works very well - just using manual classification for the manuscript
# ------------------------------------------------------------------ #
def classify_feature_type(text: str) -> str:
    """Heuristic that matches the legacy categories."""
    t = str(text).lower()

    history = "history:" in t
    sign    = ("sign:" in t) or ("symptom" in t)
    score   = any(k in t for k in ("score", "points", "rule"))
    test    = any(k in t for k in ("test:", "lab", "troponin", "d‑dimer"))
    img     = any(k in t for k in (
        "ultrasound", "ct", "mri", "x‑ray", "radiograph", "imaging",
        "echo", "angiogram"))

    if history and test: return "History and Test"
    if history and img:  return "History and imaging"
    if history:          return "History_"
    if sign:             return "Sign_symptom"
    if score:            return "Score"
    if test:             return "Test finding"
    if img:              return "Imaging finding"
    return "Diagnosis"

# ------------------------------------------------------------------ #
# 2.  Converter – writes **one** sheet (same name as in template)     #
# ------------------------------------------------------------------ #
def convert_lr_workbook(
    *,                       # keyword‑only for clarity
    input_file: str | Path,  # e.g. "nnt_lrs_with_estimated.xlsx"
    template_file: str | Path,  # updated example workbook
    output_file: str | Path = "nnt_lrs_converted.xlsx",
) -> None:
    """
    • Collapses all per‑condition tabs from `input_file` into a master frame.
    • Adds the right‑most `condition` column (original tab name).
    • Writes a single worksheet whose name matches the (only) sheet
      found in `template_file`, with the header row stored as **row 1**
      (no Excel column headers) to preserve downstream‑notebook compatibility.
    """
    input_file    = Path(input_file)
    template_file = Path(template_file)
    output_file   = Path(output_file)

    # -------- determine the sole sheet name from the template --------------
    template_xls = pd.ExcelFile(template_file, engine="openpyxl")
    if len(template_xls.sheet_names) != 1:
        raise ValueError(
            f"Template has {len(template_xls.sheet_names)} sheets; "
            "expected exactly one after pruning."
        )
    target_sheet = template_xls.sheet_names[0]  # e.g. "Master"

    # ------------------- build master dataframe ----------------------------
    in_xls = pd.ExcelFile(input_file, engine="openpyxl")
    frames = []

    for tab in in_xls.sheet_names:
        raw = pd.read_excel(in_xls, sheet_name=tab, header=None)

        # Expect: row‑0 = diagnosis, row‑1 = column labels, row‑2+ = data
        if raw.shape[0] < 3:
            continue  # skip empty or malformed tabs

        header = raw.iloc[1]
        df = raw.iloc[2:].copy()
        df.columns = header
        df = df.loc[:, ~df.columns.isna()]  # drop unnamed columns

        # back‑fill Feature Type if the sheet didn't have it
        if "Feature Type" not in df.columns:
            df["Feature Type"] = df.iloc[:, 0].apply(classify_feature_type)

        df["condition"] = tab          # new right‑most column
        frames.append(df)

    if not frames:
        raise ValueError("No valid data found in the input workbook.")

    master = pd.concat(frames, ignore_index=True)
    # ensure 'condition' is the final column
    master = master[[c for c in master.columns if c != "condition"] + ["condition"]]

    # ------------------------ write single sheet ---------------------------
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        # replicate legacy layout: header row lives **inside** the block
        block = pd.concat(
            [pd.DataFrame([master.columns], columns=master.columns), master],
            ignore_index=True,
        )
        block.to_excel(
            writer,
            sheet_name=target_sheet[:31],  # Excel has a 31‑char cap
            index=False,
            header=False,
        )

    print(f"Converted workbook written to: {output_file} "
          f"(single sheet: '{target_sheet}')")
    
# Run the conversion
convert_lr_workbook(
    input_file   = 'nnt_lrs_with_estimated.xlsx',   # produced after LR loop
    template_file= 'example_template.xlsx',  # updated example
    output_file  = 'NNT_LRs_08-26-2025.xlsx',
)